# Access reconstructed event data from ATLAS

This section assumes you've read through the introductory information in [notebook 1a](1a-Introduction-information.ipynb) - if that page doesn't work the info is also available in pdf form [here](images/1a-Introduction-information.pdf) (right-click and save in case it doesn't open in your browser).
___________________________________________________________

## Data analysis with ROOT
We import the library ROOT which comes with tools for analysing and visualising the data. ROOT is written in C++ but a python interface allows convenient use of it from a python interpreter environment.

Remember you can execute the code cells by pressing `Shift+Enter`.

The first box below installs ROOT in case you're working with Google Colab.

In [ ]:
# install the ROOT libraries if you're using Google Colab - this usually takes a minute or two
# if not we assume we're running in the docker container where it is already available
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB:
    # install ROOT and update a few other packages
    !wget https://github.com/MohamedElashri/ROOT/releases/download/ubuntu/root_v6.28.04_Ubuntu_20.04.zip
    !unzip /content/root_v6.28.04_Ubuntu_20.04.zip
    !apt-get install git dpkg-dev cmake g++ gcc binutils libx11-dev libxpm-dev libxft-dev libxext-dev tar gfortran subversion
    #!apt-get install libpython3.6-dev # seems to not be needed
    # the following is needed because colab upgraded the openssl library
    !wget http://archive.ubuntu.com/ubuntu/pool/main/o/openssl/libssl1.1_1.1.1f-1ubuntu2_amd64.deb
    !sudo dpkg -i libssl1.1_1.1.1f-1ubuntu2_amd64.deb
    # add some paths etc so that libraries can be found
    import sys
    sys.path.append("/content/root_build/")
    sys.path.append("/content/root_build/bin/")
    sys.path.append("/content/root_build/include/")
    sys.path.append("/content/root_build/lib/")
    import ctypes
    ctypes.cdll.LoadLibrary('/content/root_build/lib//libCore.so')
    ctypes.cdll.LoadLibrary('/content/root_build/lib//libThread.so')
    ctypes.cdll.LoadLibrary('/content/root_build/lib//libTreePlayer.so')
    print("Done installing ROOT - check that there were no error messages above!")
else:
    print("Not running the notebook in Google Colab, so assuming ROOT is already available")

In [ ]:
# before using ROOT, it needs to be imported
import ROOT

### Our first event loop
We first download ATLAS open data from the web and load it into a ROOT _tree_. The tree is a data structure which holds the variables representing the particles and measured detector signals in an event in different _branches_. Each event is stored in a separate _entry_ that can be loaded into the tree.
We start by looking at simulated data which contains events where a $Z^0$ boson decays to electron-positron pair.

In [ ]:
# Open a file with simulated data, containing events with Z bosons decaying to electron-positron pairs (hence the "Zee" in the file name)
f = ROOT.TFile.Open("https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/exactly2lep/MC/mc_361106.Zee.exactly2lep.root")
tree = f.Get("mini") # get the tree (called "mini"). Every "row" in a tree is an event
print("The loaded tree contains %d events" % tree.GetEntries())

#### Tree content (what data is available?)
Before creating a loop where several events are processed sequentially, let's first look at what data is stored for each event. The branches in the tree called "mini" representing variables of the event are listed below. The value of a variable is retrieved with `tree.<branch_name>`. There are quite a few variables available, and it's unlikely you'll need to use most of them. Depending on what process you're interested in, you might want to use e.g. electrons and/or muons and then the `lep_*` will be of primary interest. There are also variables for photons named `ph_*`, etc (not listed below). To get a complete list of branches (variables) in the tree, you can execute the following line:

```
# ask the tree for a list of all its branches, and print their names
for b in tree.GetListOfBranches():
    print(b.GetName())
```

 <font size="3"><b>branch name</b></font> | <font size="3"><b>type</b></font> | <font size="3"><b>description</b></font> |
 ------------- | :-------------: | :-----: |
 <b id="runNumber">runNumber</b>               | int           | runNumber |
 <b id="eventNumber">eventNumber</b>           | int           | eventNumber |
 <b id="channelNumber">channelNumber</b>       | int           | channelNumber ||
 <b id="mcWeight">mcWeight</b>                 | float         | weight of an MC event |
 <b id="pvxp_n">pvxp_n</b>                     | int           | number of primary vertices |
 <b id="scaleFactor">scaleFactor</b>           | float         | overall scale factor for the preselected event |
 <b id="trigE">trigE</b>                       | bool          | boolean whether a standard trigger has fired in the egamma stream |
 <b id="trigM">trigM</b>                       | bool          | boolean whether a standard trigger has fired in the muon stream |
 <b id="passGRL">passGRL</b>                   | bool          | signifies whether event passes the GRL may be put in isGoodEvent |
 <b id="lep_n">lep_n</b>                       | int           | number of preselected leptons |
 <b id="lep_truthMatched">lep_truthMatched</b> | vector<bool>  | boolean indicating whether the lepton is matched to a truth lepton |
 <b id="lep_trigMatched">lep_trigMatched</b>   | vector<bool>  | boolean signifying whether the lepton is the one triggering the event |
 <b id="lep_pt">lep_pt</b>                    | vector<float> | transverse momentum of the lepton |
 <b id="lep_eta">lep_eta</b>                  | vector<float> | pseudo-rapidity of the lepton |
 <b id="lep_phi">lep_phi</b>                  | vector<float> | azimuthal angle of the lepton |
 <b id="lep_E">lep_E</b>                      | vector<float> | energy of the lepton |
 <b id="lep_z0">lep_z0</b>                    | vector<float> | z-coordinate of the track associated to the lepton wrt. the primary vertex |
 <b id="lep_charge">lep_charge</b>            | vector<float> | charge of the lepton |
 <b id="lep_isTight">lep_isTight</b>          | vector<bool>  | boolean indicating whether the lepton is of tight quality |
 <b id="lep_flag">lep_flag</b>                | vector<int>   | bitmask implementing object cuts of the top group |
 <b id="lep_type">lep_type</b>                | vector<int>   | number signifying the lepton type (e, mu, tau) of the lepton |
 <b id="lep_ptcone30">lep_ptcone30</b>        | vector<float> | ptcone30 isolation for the lepton |
 <b id="lep_etcone20">lep_etcone20</b>        | vector<float> | etcone20 isolation for the lepton |
 <b id="lep_trackd0pvunbiased">lep_trackd0pvunbiased</b> | vector<float>  | d0 of the track associated to the lepton at the point of closest approach (p.o.a.) |
 <b id="lep_tracksigd0pvunbiased">lep_tracksigd0pvunbiased</b> | vector<float>  | d0 signifcance of the track associated to the lepton at the p.o.a. |
 <b id="met_et">met_et                        | float         | Transverse energy of the missing momentum vector |
 <b id="met_phi">met_phi                      | float         | Azimuthal angle of the missing momentum vector |
 <b id="jet_n">jet_n                          | int           | number of selected jets |
 <b id="jet_pt">jet_pt                        | vector<float> | transverse momentum of the jet |
 <b id="jet_eta">jet_eta                      | vector<float> | pseudorapidity of the jet |
 <b id="jet_phi">jet_phi                      | vector<float> | azimuthal angle of the jet |
 <b id="jet_E">jet_E                          | vector<float> | energy of the jet |
 <b id="jet_m">jet_m                          | vector<float> | invariant mass of the jet |
 <b id="jet_jvf">jet_jvf                      | vector<float> | JetVertexFraction of the jet |
 <b id="jet_flag">jet_flag                    | vector<int>   | bitmask implementing object cuts of the top group |
 <b id="jet_trueflav">jet_trueflav            | vector<int>   | true flavor of the jet |
 <b id="jet_truthMatched">jet_truthMatched    | vector<int>   | information whether the jet matches a jet on truth level |
 <b id="jet_SV0">jet_SV0                      | vector<float> | SV0 weight of the jet |
 <b id="jet_MV1">jet_MV1                      | vector<float> | MV1 weight of the jet |
 <b id="scaleFactor_BTAG">scaleFactor_BTAG    | float         | scalefactor for btagging |
 <b id="scaleFactor_ELE">scaleFactor_ELE      | float         | scalefactor for electron efficiency |
 <b id="scaleFactor_JVFSF">scaleFactor_JVFSF  | float         | scalefactor for jet vertex fraction |
 <b id="scaleFactor_MUON">scaleFactor_MUON    | float         | scalefactor for muon efficiency |
 <b id="scaleFactor_PILEUP">scaleFactor_PILEUP   | float      | scalefactor for pileup reweighting |
 <b id="scaleFactor_TRIGGER">scaleFactor_TRIGGER | float      | scalefactor for trigger |
 <b id="scaleFactor_ZVERTEX">scaleFactor_ZVERTEX | float      | scalefactor for z-vertex reweighting |

**Note:** Another way to loop over the events in a tree is to write 

```
for evt in tree: 
   evt.theBranchYouWant...
``` 

From here on this way is used.

#### Let's try to draw the distribution for the transverse momenta of the measured electrons in the first 1000 events in the file with simulated $Z \rightarrow e^+e^-$ events that we loaded.

For more info about how the histogram class `TH1F` in ROOT works, see [documentation for TH1](https://root.cern.ch/doc/master/classTH1.html).

In [ ]:
# create a histogram with 15 bins, ranging from 15 to 90 GeV
# h_pt is the histogram object
# "pthistogram" is the internal name of this object
# the last string sets the title of the histogram and the x and y axes, respectively
h_pt = ROOT.TH1F("pthistogram", "Electron pT; Electron transverse momentum [GeV]; Number of electrons", 15, 15., 90.) 
nevents = 1000
ievt = 0
for evt in tree: # loop over the events
    ievt += 1
    if ievt >= nevents: break # terminate the loop after 1000 events
        
    for ilep in range(evt.lep_n):  # loop over leptons (lepton means electron or muon)
        if evt.lep_type[ilep] == 11: # consider only electrons (type 11)
            h_pt.Fill(evt.lep_pt[ilep]*1e-3) # fill histogram (file stores momentum in MeV)

In [ ]:
# open a canvas to draw the histogram on
canvas = ROOT.TCanvas("Canvas", "Title", 800, 600)
# draw the histogram
h_pt.Draw("e1") # draw option "e1" will draw error bars
# open the canvas
canvas.Draw()

# NB! If you run some ROOT commands - like the one above making the canvas - several times, you will 
# see a warning message about deleting an object with the same internal ROOT same name. This is just 
# because ROOT keeps track of objects it has created by their names, so generally these can safely 
# be ignored in these exercises.

<!--
<b> NB! </b> The drawn figure is interactive - try clicking around (also right-click) and scroll when hovering over it. You can change the style and color of the histogram, the title etc. -->

The figure can be saved in png format by right-clicking it, or as a vector-based pdf format through the a line of code like

`canvas.SaveAs("figure.pdf")`

(The file will be stored in the docker container, so you can download it from the directory overview where you found the various notebooks.)
___________________________________________________

#### In the next notebook you'll learn how to use the ROOT libraries to visualize the data and fit a mathematical model to measured distributions to determine the model parameters. Move on to [2-Fitting-with-ROOT](2-Fitting-with-ROOT.ipynb)!
________________________________________________________